# 취향 분석

In [1]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

from hyperopt import fmin, hp, tpe, Trials
from scipy.io import mmwrite

C:\Users\SSAFY\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
category_top_cloth_column = ['newClothId', 'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
        'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers']
category_outer_cloth_column = ['newClothId', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers']
category_pants_cloth_column = ['newClothId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers']
category_onepiece_cloth_column = ['newClothId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress']
category_skirt_cloth_column = ['newClothId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt']
category_top_user_column = ['userId', 'smallCategoryHalfshort', 'smallCategoryShirt',
       'smallCategoryCollar', 'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit', 'smallCategoryLong', 'smallCategoryShort',
       'smallCategoryOthers']
category_outer_user_column = ['userId', 
       'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
        'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers']
category_pants_user_column = ['userId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks', 'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers']
category_onepiece_user_column = ['userId', 'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress']
category_skirt_user_column = ['userId', 'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt']

In [4]:
transaction = pd.read_json('./transaction_top.json')
cloth_meta = pd.read_json('./final_top.json')
user_meta = pd.read_json('./user_top.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [5]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(281090, 3)

In [6]:
user_top_list = pd.DataFrame(transaction.userId.unique())
cloth_top_list = pd.DataFrame(transaction.newClothId.unique())
print(len(user_top_list))
print(len(cloth_top_list))
user_top_list.to_json('user_top_list.json', orient='records', force_ascii=False)
cloth_top_list.to_json('cloth_top_list.json', orient='records', force_ascii=False)

131315
59819


In [7]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [8]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [9]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [10]:
user_meta = user_meta[category_top_user_column]

In [11]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers
0,858064,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0
1,858065,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,858067,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0
3,858070,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,858071,0.400000,0.000000,0.0,0.000000,0.000000,0.400000,0.200000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
131310,1416935,0.214286,0.000000,0.0,0.071429,0.214286,0.285714,0.214286,0.0,0.0
131311,1416936,0.000000,0.333333,0.0,0.000000,0.000000,0.666667,0.000000,0.0,0.0
131312,1416938,0.000000,0.000000,0.0,0.400000,0.400000,0.200000,0.000000,0.0,0.0
131313,1416946,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0


In [12]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('smallCategoryHalfshort')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryShirt')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryCollar')] = 0.4
new_user.loc[new_user_idx, ('smallCategoryHoody')] = 0
new_user.loc[new_user_idx, ('smallCategorySweatshirt')] = 0
new_user.loc[new_user_idx, ('smallCategoryKnit')] = 0
new_user.loc[new_user_idx, ('smallCategoryLong')] = 0
new_user.loc[new_user_idx, ('smallCategoryShort')] = 0
new_user.loc[new_user_idx, ('smallCategoryOthers')] = 0
new_user = new_user[['userId',  'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers']]
new_user

,userId,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar']


In [14]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [15]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [16]:
sub_col = list(sub_user.columns)[1:]

In [17]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [18]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[904030, 871998, 1214425]


In [19]:
cloth_meta = cloth_meta[category_top_cloth_column]

In [20]:
cloth_meta

,newClothId,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers
0,1929966,1,0,0,0,0,0,0,0,0
1,1927658,1,0,0,0,0,0,0,0,0
2,1927660131403,1,0,0,0,0,0,0,0,0
3,1928209,1,0,0,0,0,0,0,0,0
4,1927270,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
249601,1413008,1,0,0,0,0,0,0,0,0
249602,779965,0,1,0,0,0,0,0,0,0
249603,1353489,1,0,0,0,0,0,0,0,0
249604,1607233,0,0,0,0,0,0,1,0,0


In [21]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [22]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

59819

In [23]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [24]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [25]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [26]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 131315
clothId의 유니크 개수 : 59819
user테이블의 userId의 유니크 개수 : 131315
cloth테이블의 clothId의 유니크 개수 : 59819


In [27]:
cloth_meta

,newClothId,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,idx
0,1440513131686,1,0,0,0,0,0,0,0,0,50031
1,1930909131765,0,0,0,0,0,0,1,0,0,7019
2,1930909131764,0,0,0,0,0,0,1,0,0,7840
3,1930909131766,0,0,0,0,0,0,1,0,0,7374
4,1931411131771,1,0,0,0,0,0,0,0,0,9496
...,...,...,...,...,...,...,...,...,...,...,...
59814,1281638267701,0,0,0,1,0,0,0,0,0,31274
59815,1622670267773,0,1,0,0,0,0,0,0,0,9458
59816,1556851267795,0,0,0,0,0,1,0,0,0,47549
59817,1651849267836,0,0,0,0,0,1,0,0,0,3995


In [28]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['smallCategoryHalfshort'][i], user_meta['smallCategoryShirt'][i], user_meta['smallCategoryCollar'][i],
                        user_meta['smallCategoryHoody'][i], user_meta['smallCategorySweatshirt'][i], user_meta['smallCategoryKnit'][i],
                        user_meta['smallCategoryLong'][i], user_meta['smallCategoryShort'][i], user_meta['smallCategoryOthers'][i]]) for i in range(user_meta.shape[0])]

In [29]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['smallCategoryHalfshort'][i], cloth_meta['smallCategoryShirt'][i], cloth_meta['smallCategoryCollar'][i],
                        cloth_meta['smallCategoryHoody'][i], cloth_meta['smallCategorySweatshirt'][i], cloth_meta['smallCategoryKnit'][i],
                        cloth_meta['smallCategoryLong'][i], cloth_meta['smallCategoryShort'][i], cloth_meta['smallCategoryOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [30]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [31]:
interactions, weights = dataset.build_interactions(transaction_source)

In [32]:
user_features = dataset.build_user_features(user_features_source)

In [33]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [34]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [35]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [36]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [37]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 15, lrn_rate: 0.08011, precision: 0.00319                                                                     
no_comp: 10, lrn_rate: 0.07910, precision: 0.00327                                                                     
no_comp: 10, lrn_rate: 0.09059, precision: 0.00318                                                                     
no_comp: 10, lrn_rate: 0.06772, precision: 0.00321                                                                     
no_comp: 20, lrn_rate: 0.08772, precision: 0.00266                                                                     
no_comp: 10, lrn_rate: 0.07315, precision: 0.00305                                                                     
no_comp: 20, lrn_rate: 0.09748, precision: 0.00289                                                                     
no_comp: 15, lrn_rate: 0.06291, precision: 0.00311                                                                     
no_comp: 15, lrn_rate: 0.06201, precisio

In [38]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.062 1


In [39]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.91it/s]


In [40]:
# 모델 저장하기
import pickle
with open('model_category_top.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [40]:
# 모델 불러오기
import pickle
with open('model_category_top.pickle', 'rb') as f:
    model_top = pickle.load(f)

In [41]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [42]:
rec_size_top = sample_recommendation(model_top, cloth_meta.idx, real_user)
rec_size_top

[1440587133960,
 1282892131862,
 2139133140300,
 2152337133146,
 1648578142971,
 1803785131756,
 1804104134824,
 1931904131835,
 1572660131416,
 2139144142723]

In [45]:
real_user

[904030, 871998, 1214425]

In [46]:
cloth_meta.insert(11, 'what', 16, True)
user_meta.insert(10, 'what', 16, True)

In [47]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,1440513131686,50031,16
1,1930909131765,7019,16
2,1930909131764,7840,16
3,1930909131766,7374,16
4,1931411131771,9496,16
...,...,...,...
59814,1281638267701,31274,16
59815,1622670267773,9458,16
59816,1556851267795,47549,16
59817,1651849267836,3995,16


In [48]:
cloth_meta.to_json('category_top_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('category_top_user_meta.json', orient='records', force_ascii=False)

In [49]:
user_meta

,userId,smallCategoryHalfshort,smallCategoryShirt,smallCategoryCollar,smallCategoryHoody,smallCategorySweatshirt,smallCategoryKnit,smallCategoryLong,smallCategoryShort,smallCategoryOthers,what
0,858064,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.666667,0.0,0.0,16
1,858065,0.500000,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,16
2,858067,0.250000,0.000000,0.0,0.500000,0.250000,0.000000,0.000000,0.0,0.0,16
3,858070,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,16
4,858071,0.400000,0.000000,0.0,0.000000,0.000000,0.400000,0.200000,0.0,0.0,16
...,...,...,...,...,...,...,...,...,...,...,...
131310,1416935,0.214286,0.000000,0.0,0.071429,0.214286,0.285714,0.214286,0.0,0.0,16
131311,1416936,0.000000,0.333333,0.0,0.000000,0.000000,0.666667,0.000000,0.0,0.0,16
131312,1416938,0.000000,0.000000,0.0,0.400000,0.400000,0.200000,0.000000,0.0,0.0,16
131313,1416946,0.166667,0.000000,0.0,0.166667,0.166667,0.166667,0.333333,0.0,0.0,16


# Outer

In [41]:
transaction = pd.read_json('./transaction_outer.json')
cloth_meta = pd.read_json('./final_outer.json')
user_meta = pd.read_json('./user_outer.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [42]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(171253, 3)

In [43]:
user_outer_list = pd.DataFrame(transaction.userId.unique())
cloth_outer_list = pd.DataFrame(transaction.newClothId.unique())
print(len(user_outer_list))
print(len(cloth_outer_list))
user_outer_list.to_json('user_outer_list.json', orient='records', force_ascii=False)
cloth_outer_list.to_json('cloth_outer_list.json', orient='records', force_ascii=False)

93511
26877


In [44]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [45]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [46]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [47]:
user_meta = user_meta[category_outer_user_column]

In [48]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers
0,488503,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
1,488504,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
2,488506,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,488507,0.0,0.000000,0.000000,0.666667,0.0,0.0,0.0,0.0,0.0,0.333333,0.0
4,488509,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
93245,858056,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.4,0.0,0.600000,0.0
93246,858057,0.0,0.333333,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
93247,858058,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
93248,858060,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.000000,0.0


In [49]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('smallCategoryHoodie')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryBlouson')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryRiders')] = 0.4
new_user.loc[new_user_idx, ('smallCategoryMustang')] = 0
new_user.loc[new_user_idx, ('smallCategoryCardigan')] = 0
new_user.loc[new_user_idx, ('smallCategoryFleece')] = 0
new_user.loc[new_user_idx, ('smallCategoryCoat')] = 0
new_user.loc[new_user_idx, ('smallCategoryPaddedcoat')] = 0
new_user.loc[new_user_idx, ('smallCategoryVest')] = 0
new_user.loc[new_user_idx, ('smallCategoryJacket')] = 0
new_user.loc[new_user_idx, ('smallCategoryOthers')] = 0
new_user = new_user[['userId', 'smallCategoryHoodie', 'smallCategoryBlouson',
       'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan',
       'smallCategoryFleece', 'smallCategoryCoat', 'smallCategoryPaddedcoat',
       'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers']]
new_user

,userId,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders']


In [51]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [52]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [53]:
sub_col = list(sub_user.columns)[1:]

In [54]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [55]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[604845, 842621, 515560]


In [56]:
cloth_meta = cloth_meta[category_outer_cloth_column]

In [57]:
cloth_meta

,newClothId,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,smallCategoryJacket,smallCategoryOthers
0,2401813,0,0,0,0,0,0,0,0,0,1,0
1,234005278848,0,0,0,0,1,0,0,0,0,0,0
2,2379678,0,1,0,0,0,0,0,0,0,0,0
3,2360803,0,0,0,0,0,0,0,1,0,0,0
4,2354237,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
85636,2322135130506,0,0,0,0,0,0,0,0,0,1,0
85637,2322135130507,0,0,0,0,0,0,0,0,0,1,0
85638,2322135130508,0,0,0,0,0,0,0,0,0,1,0
85639,2352578,0,1,0,0,0,0,0,0,0,0,0


In [58]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [59]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

25916

In [60]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [61]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [62]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [63]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 93250
clothId의 유니크 개수 : 25916
user테이블의 userId의 유니크 개수 : 93250
cloth테이블의 clothId의 유니크 개수 : 25916


In [64]:
user_meta.columns

Index(['userId', 'smallCategoryHoodie', 'smallCategoryBlouson',
       'smallCategoryRiders', 'smallCategoryMustang', 'smallCategoryCardigan',
       'smallCategoryFleece', 'smallCategoryCoat', 'smallCategoryPaddedcoat',
       'smallCategoryVest', 'smallCategoryJacket', 'smallCategoryOthers'],
      dtype='object')

In [65]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['smallCategoryHoodie'][i], user_meta['smallCategoryBlouson'][i], user_meta['smallCategoryRiders'][i],
                        user_meta['smallCategoryMustang'][i], user_meta['smallCategoryCardigan'][i], user_meta['smallCategoryFleece'][i],
                        user_meta['smallCategoryCoat'][i], user_meta['smallCategoryPaddedcoat'][i], user_meta['smallCategoryVest'][i],
                        user_meta['smallCategoryJacket'][i], user_meta['smallCategoryOthers'][i]]) for i in range(user_meta.shape[0])]

In [66]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['smallCategoryHoodie'][i], cloth_meta['smallCategoryBlouson'][i], cloth_meta['smallCategoryRiders'][i],
                        cloth_meta['smallCategoryMustang'][i], cloth_meta['smallCategoryCardigan'][i], cloth_meta['smallCategoryFleece'][i],
                        cloth_meta['smallCategoryCoat'][i], cloth_meta['smallCategoryPaddedcoat'][i], cloth_meta['smallCategoryVest'][i],
                        cloth_meta['smallCategoryJacket'][i], cloth_meta['smallCategoryOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [67]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [68]:
interactions, weights = dataset.build_interactions(transaction_source)

In [69]:
user_features = dataset.build_user_features(user_features_source)

In [70]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [71]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [72]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [73]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [74]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 20, lrn_rate: 0.05732, precision: 0.00526                                                                     
no_comp: 10, lrn_rate: 0.06129, precision: 0.00578                                                                     
no_comp: 10, lrn_rate: 0.09980, precision: 0.00558                                                                     
no_comp: 20, lrn_rate: 0.06102, precision: 0.00576                                                                     
no_comp: 20, lrn_rate: 0.09003, precision: 0.00541                                                                     
no_comp: 20, lrn_rate: 0.06343, precision: 0.00531                                                                     
no_comp: 10, lrn_rate: 0.09975, precision: 0.00562                                                                     
no_comp: 15, lrn_rate: 0.09506, precision: 0.00494                                                                     
no_comp: 20, lrn_rate: 0.08807, precisio

In [31]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.05 2


In [75]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.11it/s]


In [76]:
# 모델 저장하기
import pickle
with open('model_category_outer.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [84]:
# 모델 불러오기
import pickle
with open('model_category_outer.pickle', 'rb') as f:
    model_outer = pickle.load(f)

In [85]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [86]:
rec_size_top = sample_recommendation(model_outer, cloth_meta.idx, real_user)
rec_size_top

[157135683282,
 238071082834,
 231905785481,
 117337981361,
 1577133111047,
 208840683451,
 132418381736,
 208155079212,
 177400380239,
 1736613110797]

In [87]:
real_user

[604845, 842621, 515560]

In [88]:
cloth_meta.insert(11, 'what', 17, True)
user_meta.insert(10, 'what', 17, True)

In [89]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,234005278848,25240,17
1,208155079211,20700,17
2,208155079212,15426,17
3,208155079213,23065,17
4,208155079214,1832,17
...,...,...,...
25911,2080918130502,10939,17
25912,2080918130504,7261,17
25913,2322135130506,21919,17
25914,2322135130507,20550,17


In [90]:
cloth_meta.to_json('category_outer_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('category_outer_user_meta.json', orient='records', force_ascii=False)

In [91]:
user_meta

,userId,smallCategoryHoodie,smallCategoryBlouson,smallCategoryRiders,smallCategoryMustang,smallCategoryCardigan,smallCategoryFleece,smallCategoryCoat,smallCategoryPaddedcoat,smallCategoryVest,what,smallCategoryJacket,smallCategoryOthers
0,488503,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,17,0.000000,0.0
1,488504,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,17,0.000000,0.0
2,488506,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,17,0.000000,0.0
3,488507,0.0,0.000000,0.000000,0.666667,0.0,0.0,0.0,0.0,0.0,17,0.333333,0.0
4,488509,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,17,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93245,858056,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.4,0.0,17,0.600000,0.0
93246,858057,0.0,0.333333,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,17,0.000000,0.0
93247,858058,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,17,1.000000,0.0
93248,858060,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,17,0.000000,0.0


# Pants

In [77]:
transaction = pd.read_json('./transaction_pants.json')
cloth_meta = pd.read_json('./final_pants.json')
user_meta = pd.read_json('./user_pants.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [78]:
transaction = transaction[transaction.shopCnt > 1]
transaction.shape

(273862, 3)

In [79]:
user_pants_list = pd.DataFrame(transaction.userId.unique())
cloth_pants_list = pd.DataFrame(transaction.newClothId.unique())
print(len(user_pants_list))
print(len(cloth_pants_list))
user_pants_list.to_json('user_pants_list.json', orient='records', force_ascii=False)
cloth_pants_list.to_json('cloth_pants_list.json', orient='records', force_ascii=False)

120991
37083


In [80]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [81]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [82]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [83]:
user_meta = user_meta[category_pants_user_column]

In [84]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers
0,14135,0.666667,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000
1,14141,0.407407,0.222222,0.185185,0.185185,0.000000,0.0,0.0,0.000000
2,14146,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000
3,14148,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.000000
4,14158,0.500000,0.200000,0.150000,0.150000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
115960,488479,0.423077,0.153846,0.230769,0.153846,0.000000,0.0,0.0,0.038462
115961,488483,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000
115962,488490,0.181818,0.272727,0.000000,0.272727,0.181818,0.0,0.0,0.090909
115963,488491,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000


In [85]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('smallCategoryDenimpants')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryCottonpants')] = 0.3
new_user.loc[new_user_idx, ('smallCategorySlacks')] = 0.4
new_user.loc[new_user_idx, ('smallCategoryJoggerpants')] = 0
new_user.loc[new_user_idx, ('smallCategoryShortpants')] = 0
new_user.loc[new_user_idx, ('smallCategoryLeggings')] = 0
new_user.loc[new_user_idx, ('smallCategoryJumpsuit')] = 0
new_user.loc[new_user_idx, ('smallCategoryOthers')] = 0
new_user = new_user[['userId',  'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers']]
new_user

,userId,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers
0,8299.0,0.3,0.3,0.4,0.0,0.0,0.0,0.0,0.0


In [86]:
colum = list(new_user.columns)
di = dict()
for i in range(len(colum)):
    di[colum[i]] = new_user[colum[i]][0]
col_list = []
for idx, key in enumerate(di):
    if idx > 0 and di[key] > 0.2:
        col_list.append(key)
col_list.insert(0, 'userId')
print(col_list)

['userId', 'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks']


In [87]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [88]:
for i in range(1, len(col_list)):
    sub_user[col_list[i]] = sub_user.apply(lambda x: abs(x[col_list[i]]-new_user[col_list[i]].iloc[0]), axis=1)

In [89]:
sub_col = list(sub_user.columns)[1:]

In [90]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sum(sub_user[col_i] for col_i in sub_col)
sub_user = sub_user.sort_values('favorSum')

In [91]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[295116, 454959, 79013]


In [92]:
cloth_meta = cloth_meta[category_pants_cloth_column]

In [93]:
cloth_meta

,newClothId,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers
0,1971752,0,0,0,0,0,0,0,1
1,726566,0,0,0,1,0,0,0,0
2,2213618,0,0,0,0,0,0,0,1
3,2265443,0,0,1,0,0,0,0,0
4,2005302,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
106873,1889420,1,0,0,0,0,0,0,0
106874,1275944,1,0,0,0,0,0,0,0
106875,1889504,1,0,0,0,0,0,0,0
106876,552678,0,0,0,0,1,0,0,0


In [94]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [95]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

33530

In [96]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [97]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [98]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [99]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 115965
clothId의 유니크 개수 : 33530
user테이블의 userId의 유니크 개수 : 115965
cloth테이블의 clothId의 유니크 개수 : 33530


In [100]:
user_meta.columns

Index(['userId', 'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers'],
      dtype='object')

In [101]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['smallCategoryDenimpants'][i], user_meta['smallCategoryCottonpants'][i], user_meta['smallCategorySlacks'][i],
                        user_meta['smallCategoryJoggerpants'][i], user_meta['smallCategoryShortpants'][i], user_meta['smallCategoryLeggings'][i],
                        user_meta['smallCategoryJumpsuit'][i], user_meta['smallCategoryOthers'][i]]) for i in range(user_meta.shape[0])]

In [102]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['smallCategoryDenimpants'][i], cloth_meta['smallCategoryCottonpants'][i], cloth_meta['smallCategorySlacks'][i],
                        cloth_meta['smallCategoryJoggerpants'][i], cloth_meta['smallCategoryShortpants'][i], cloth_meta['smallCategoryLeggings'][i],
                        cloth_meta['smallCategoryJumpsuit'][i], cloth_meta['smallCategoryOthers'][i]]) for i in range(cloth_meta.shape[0])]

In [103]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [104]:
interactions, weights = dataset.build_interactions(transaction_source)

In [105]:
user_features = dataset.build_user_features(user_features_source)

In [106]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [107]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [108]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [109]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [110]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 15, lrn_rate: 0.09627, precision: 0.00533                                                                     
no_comp: 10, lrn_rate: 0.06310, precision: 0.00525                                                                     
no_comp: 10, lrn_rate: 0.08575, precision: 0.00533                                                                     
no_comp: 15, lrn_rate: 0.09512, precision: 0.00537                                                                     
no_comp: 10, lrn_rate: 0.06560, precision: 0.00535                                                                     
no_comp: 20, lrn_rate: 0.08718, precision: 0.00494                                                                     
no_comp: 15, lrn_rate: 0.06641, precision: 0.00480                                                                     
no_comp: 20, lrn_rate: 0.08875, precision: 0.00498                                                                     
no_comp: 20, lrn_rate: 0.05384, precisio

In [111]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.095 1


In [112]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.45it/s]


In [113]:
# 모델 저장하기
import pickle
with open('model_category_pants.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [127]:
# 모델 불러오기
import pickle
with open('model_category_pants.pickle', 'rb') as f:
    model_pants = pickle.load(f)

In [128]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [129]:
rec_size_top = sample_recommendation(model_pants, cloth_meta.idx, real_user)
rec_size_top

[19409615513,
 214926918340,
 21801229810,
 12086734993,
 141285112055,
 166869912945,
 17611826560,
 174162315738,
 196639011167,
 23422909921]

In [130]:
real_user

[295116, 454959, 79013]

In [134]:
cloth_meta.insert(10, 'what', 18, True)
user_meta.insert(9, 'what', 18, True)

In [135]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,16225354884,11720,18
1,14988915210,15601,18
2,19409615513,29005,18
3,20193975542,22897,18
4,18466875743,17729,18
...,...,...,...
33525,180323169094,6877,18
33526,182037869103,19148,18
33527,194119769137,18084,18
33528,179115969145,16294,18


In [136]:
cloth_meta.to_json('category_pants_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('category_pants_user_meta.json', orient='records', force_ascii=False)

In [137]:
user_meta

,userId,smallCategoryDenimpants,smallCategoryCottonpants,smallCategorySlacks,smallCategoryJoggerpants,smallCategoryShortpants,smallCategoryLeggings,smallCategoryJumpsuit,smallCategoryOthers,what
0,14135,0.666667,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000,18
1,14141,0.407407,0.222222,0.185185,0.185185,0.000000,0.0,0.0,0.000000,18
2,14146,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,18
3,14148,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.000000,18
4,14158,0.500000,0.200000,0.150000,0.150000,0.000000,0.0,0.0,0.000000,18
...,...,...,...,...,...,...,...,...,...,...
115960,488479,0.423077,0.153846,0.230769,0.153846,0.000000,0.0,0.0,0.038462,18
115961,488483,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,18
115962,488490,0.181818,0.272727,0.000000,0.272727,0.181818,0.0,0.0,0.090909,18
115963,488491,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,18


# Onepiece

In [138]:
transaction = pd.read_json('./transaction_onepiece.json')
cloth_meta = pd.read_json('./final_onepiece.json')
user_meta = pd.read_json('./user_onepiece.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [139]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [140]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [141]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [142]:
user_meta = user_meta[category_onepiece_user_column]

In [143]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress
0,8297,1.0,0.0,0.0
1,8298,1.0,0.0,0.0
2,8299,0.0,1.0,0.0
3,8300,1.0,0.0,0.0
4,8301,0.0,0.0,1.0
...,...,...,...,...
5832,14129,0.0,1.0,0.0
5833,14130,1.0,0.0,0.0
5834,14131,0.5,0.5,0.0
5835,14132,1.0,0.0,0.0


In [144]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('smallCategoryMinidress')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryMidi')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryMaxidress')] = 0.4
new_user = new_user[['userId',  'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress']]
new_user

,userId,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress
0,8299.0,0.3,0.3,0.4


In [158]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [159]:
sub_user.smallCategoryMinidress = sub_user.apply(lambda x: abs(x.smallCategoryMinidress-new_user.smallCategoryMinidress.iloc[0]), axis=1)
sub_user.smallCategoryMidi = sub_user.apply(lambda x: abs(x.smallCategoryMidi-new_user.smallCategoryMidi.iloc[0]), axis=1)
sub_user.smallCategoryMaxidress = sub_user.apply(lambda x: abs(x.smallCategoryMaxidress-new_user.smallCategoryMaxidress.iloc[0]), axis=1)

In [160]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sub_user.smallCategoryMinidress+sub_user.smallCategoryMidi+sub_user.smallCategoryMaxidress
sub_user = sub_user.sort_values('favorSum')

In [161]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[13651, 8841, 9688]


In [165]:
cloth_meta = cloth_meta[category_onepiece_cloth_column]

In [166]:
cloth_meta

,newClothId,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress
0,1475941,0,1,0
1,1995082,0,0,1
2,1723870,0,1,0
3,2197982,0,1,0
4,2247825,0,0,1
...,...,...,...,...
9870,2324195,0,1,0
9871,14649944700,0,0,1
9872,2352427,0,1,0
9873,1457626,1,0,0


In [167]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [168]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2047

In [169]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [170]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [171]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [172]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 5837
clothId의 유니크 개수 : 2047
user테이블의 userId의 유니크 개수 : 5837
cloth테이블의 clothId의 유니크 개수 : 2047


In [173]:
user_meta

,userId,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress
0,8297,1.0,0.0,0.0
1,8298,1.0,0.0,0.0
2,8299,0.0,1.0,0.0
3,8300,1.0,0.0,0.0
4,8301,0.0,0.0,1.0
...,...,...,...,...
5832,14129,0.0,1.0,0.0
5833,14130,1.0,0.0,0.0
5834,14131,0.5,0.5,0.0
5835,14132,1.0,0.0,0.0


In [174]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['smallCategoryMinidress'][i], user_meta['smallCategoryMidi'][i], user_meta['smallCategoryMaxidress'][i]]) for i in range(user_meta.shape[0])]

In [175]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['smallCategoryMinidress'][i], cloth_meta['smallCategoryMidi'][i], cloth_meta['smallCategoryMaxidress'][i]]) for i in range(cloth_meta.shape[0])]

In [176]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [177]:
interactions, weights = dataset.build_interactions(transaction_source)

In [178]:
user_features = dataset.build_user_features(user_features_source)

In [179]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [180]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [181]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [182]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [183]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 10, lrn_rate: 0.09564, precision: 0.00773    
no_comp: 20, lrn_rate: 0.05525, precision: 0.01010                                 
no_comp: 15, lrn_rate: 0.08130, precision: 0.01204                                 
no_comp: 10, lrn_rate: 0.06293, precision: 0.00624                                 
no_comp: 15, lrn_rate: 0.05524, precision: 0.01159                                
no_comp: 20, lrn_rate: 0.06829, precision: 0.00966                                
no_comp: 15, lrn_rate: 0.07060, precision: 0.00981                                
no_comp: 15, lrn_rate: 0.08503, precision: 0.01085                                
no_comp: 10, lrn_rate: 0.09954, precision: 0.00698                                
no_comp: 10, lrn_rate: 0.06966, precision: 0.00579                                
100%|██████████| 10/10 [00:03<00:00,  3.26trial/s, best loss: -0.01203566137701273]


In [184]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.081 1


In [185]:
learning_rate = 0.03
no_components = 3

In [186]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 70.66it/s]


In [187]:
# 모델 저장하기
import pickle
with open('model_category_onepiece.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [188]:
# 모델 불러오기
import pickle
with open('model_category_onepiece.pickle', 'rb') as f:
    model_onepiece = pickle.load(f)

In [189]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [190]:
rec_size_top = sample_recommendation(model_onepiece, cloth_meta.idx, real_user)
rec_size_top

[22207502998,
 20163412966,
 17093152675,
 14435003027,
 19345122958,
 21430112964,
 19186162758,
 19186162757,
 19173573021,
 21101903074]

In [191]:
real_user

[13651, 8841, 9688]

In [194]:
cloth_meta.insert(5, 'what', 19, True)
user_meta.insert(4, 'what', 19, True)

In [195]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,14094532674,279,19
1,14163822698,1239,19
2,18962342742,78,19
3,14829532807,1135,19
4,13617652829,1543,19
...,...,...,...
2042,20653784693,298,19
2043,13105704695,1719,19
2044,14406794698,837,19
2045,19412954699,1421,19


In [196]:
cloth_meta.to_json('category_onepiece_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('category_onepiece_user_meta.json', orient='records', force_ascii=False)

In [197]:
user_meta

,userId,smallCategoryMinidress,smallCategoryMidi,smallCategoryMaxidress,what
0,8297,1.0,0.0,0.0,19
1,8298,1.0,0.0,0.0,19
2,8299,0.0,1.0,0.0,19
3,8300,1.0,0.0,0.0,19
4,8301,0.0,0.0,1.0,19
...,...,...,...,...,...
5832,14129,0.0,1.0,0.0,19
5833,14130,1.0,0.0,0.0,19
5834,14131,0.5,0.5,0.0,19
5835,14132,1.0,0.0,0.0,19


# Skirt

In [198]:
transaction = pd.read_json('./transaction_skirt.json')
cloth_meta = pd.read_json('./final_skirt.json')
user_meta = pd.read_json('./user_skirt.json')
transaction = transaction[['userId', 'newClothId', 'shopCnt']]

In [199]:
unique_id_cloth_meta = list(set(cloth_meta.newClothId))
transaction=transaction.loc[transaction.newClothId.isin(unique_id_cloth_meta),:]

In [200]:
a = transaction[['newClothId']]
a.insert(1, 'idx', 0, True)
a = a.drop_duplicates(['newClothId'])
a.idx = range(len(a))

In [201]:
transaction = pd.merge(transaction, a, how="left", on="newClothId")

In [202]:
user_meta = user_meta[category_skirt_user_column]

In [203]:
unique_id_user_meta = list(set(transaction.userId))
user_meta=user_meta.loc[user_meta.userId.isin(unique_id_user_meta),:]
user_meta = user_meta.reset_index(drop=True)
user_meta

,userId,smallCategoryMiniskirt,smallCategoryMidi,smallCategoryLongskirt
0,0,0.0,0.0,1.0
1,1,0.5,0.0,0.5
2,2,1.0,0.0,0.0
3,3,1.0,0.0,0.0
4,4,1.0,0.0,0.0
...,...,...,...,...
8292,8292,0.0,0.0,1.0
8293,8293,0.0,0.0,1.0
8294,8294,1.0,0.0,0.0
8295,8295,1.0,0.0,0.0


In [205]:
new_user = pd.DataFrame(user_meta.loc[0]).transpose()
new_user_idx = new_user.index
new_user.loc[new_user_idx, ('userId')] = 8299
new_user.loc[new_user_idx, ('smallCategoryMiniskirt')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryMidi')] = 0.3
new_user.loc[new_user_idx, ('smallCategoryLongskirt')] = 0.4
new_user = new_user[['userId',  'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt']]
new_user

,userId,smallCategoryMiniskirt,smallCategoryMidi,smallCategoryLongskirt
0,8299.0,0.3,0.3,0.4


In [206]:
from copy import deepcopy
sub_user = deepcopy(user_meta)

In [207]:
sub_user.smallCategoryMiniskirt = sub_user.apply(lambda x: abs(x.smallCategoryMiniskirt-new_user.smallCategoryMiniskirt.iloc[0]), axis=1)
sub_user.smallCategoryMidi = sub_user.apply(lambda x: abs(x.smallCategoryMidi-new_user.smallCategoryMidi.iloc[0]), axis=1)
sub_user.smallCategoryLongskirt = sub_user.apply(lambda x: abs(x.smallCategoryLongskirt-new_user.smallCategoryLongskirt.iloc[0]), axis=1)

In [208]:
sub_user.insert(1, 'favorSum', 0, True)
sub_user.favorSum = sub_user.smallCategoryMiniskirt+sub_user.smallCategoryMidi+sub_user.smallCategoryLongskirt
sub_user = sub_user.sort_values('favorSum')

In [209]:
real_user = []
for i in range(3):
    real_user.append(sub_user.iloc[i].userId)
real_user = [int(i) for i in real_user]
print(real_user)

[7389, 6361, 257]


In [210]:
cloth_meta = cloth_meta[category_skirt_cloth_column]

In [211]:
cloth_meta

,newClothId,smallCategoryMiniskirt,smallCategoryMidi,smallCategoryLongskirt
0,2018790,0,0,1
1,1844574,1,0,0
2,1672093,0,1,0
3,1687533,0,1,0
4,2338446,0,1,0
...,...,...,...,...
8871,21172562649,0,0,1
8872,21172562648,0,0,1
8873,23400702652,1,0,0
8874,23400702653,1,0,0


In [212]:
cloth_meta = pd.merge(cloth_meta, a, how="left", on="newClothId")

In [213]:
unique_cloth_id = list(transaction['newClothId'].unique())
len(unique_cloth_id)

2654

In [214]:
a = cloth_meta.newClothId.isin(unique_cloth_id)
cloth_meta = cloth_meta[a]
cloth_meta.index = range(len(cloth_meta))

In [215]:
cloth_meta.idx = cloth_meta.idx.astype('int')

In [216]:
transaction_source = [(transaction['userId'][i], transaction['idx'][i]) for i in range(transaction.shape[0])]

In [217]:
print('userId의 유니크 개수 :', len(transaction.userId.unique()))
print('clothId의 유니크 개수 :', len(transaction.idx.unique()))
print('user테이블의 userId의 유니크 개수 :', len(user_meta.userId.unique()))
print('cloth테이블의 clothId의 유니크 개수 :', len(cloth_meta.idx.unique()))

userId의 유니크 개수 : 8297
clothId의 유니크 개수 : 2654
user테이블의 userId의 유니크 개수 : 8297
cloth테이블의 clothId의 유니크 개수 : 2654


In [219]:
user_features_source = [(user_meta['userId'][i],
                        [user_meta['smallCategoryMiniskirt'][i], user_meta['smallCategoryMidi'][i], user_meta['smallCategoryLongskirt'][i]]) for i in range(user_meta.shape[0])]

In [220]:
cloth_features_source = [(cloth_meta['idx'][i],
                        [cloth_meta['smallCategoryMiniskirt'][i], cloth_meta['smallCategoryMidi'][i], cloth_meta['smallCategoryLongskirt'][i]]) for i in range(cloth_meta.shape[0])]

In [221]:
dataset = Dataset()
dataset.fit(users=transaction['userId'].unique(),
           items=transaction['idx'].unique(),
           user_features=user_meta[user_meta.columns[1:]].values.flatten(),
           item_features=cloth_meta[cloth_meta.columns[1:]].values.flatten())

In [222]:
interactions, weights = dataset.build_interactions(transaction_source)

In [223]:
user_features = dataset.build_user_features(user_features_source)

In [224]:
cloth_features = dataset.build_item_features(cloth_features_source)

In [225]:
train, test = random_train_test_split(interactions, test_percentage=0.2)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()
test_weights = test.multiply(weights).tocoo()

In [226]:
# no_components: 10
# learning_rate : 0.05
trials = Trials()
space = [hp.choice('no_components', range(10, 21, 5)),
         hp.uniform('learning_rate', 0.05, 0.1)]

In [227]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params
    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=cloth_features,
              user_features=user_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=cloth_features, user_features=user_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [228]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 15, lrn_rate: 0.05733, precision: 0.01111    
no_comp: 15, lrn_rate: 0.07954, precision: 0.01111                                 
no_comp: 10, lrn_rate: 0.07706, precision: 0.01111                                 
no_comp: 20, lrn_rate: 0.05411, precision: 0.00978                                 
no_comp: 10, lrn_rate: 0.06632, precision: 0.01029                                 
no_comp: 10, lrn_rate: 0.05032, precision: 0.01100                                 
no_comp: 20, lrn_rate: 0.09722, precision: 0.00968                                 
no_comp: 20, lrn_rate: 0.06064, precision: 0.01111                                 
no_comp: 15, lrn_rate: 0.08470, precision: 0.01039                                 
no_comp: 10, lrn_rate: 0.07113, precision: 0.01172                                 
100%|██████████| 10/10 [00:05<00:00,  1.86trial/s, best loss: -0.011716759763658047]


In [229]:
learning_rate = round(best_params['learning_rate'], 3)
no_components = best_params['no_components']
print(learning_rate, no_components)

0.071 0


In [230]:
learning_rate = 0.03
no_components = 3

In [231]:
model = LightFM(no_components=no_components,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=learning_rate,
                random_state=0)

model.fit(interactions=interactions,
          item_features=cloth_features,
          user_features=user_features,
          sample_weight=weights,
          epochs=10,
          verbose=True)

Epoch: 100%|██████████| 10/10 [00:00<00:00, 52.62it/s]


In [232]:
# 모델 저장하기
import pickle
with open('model_category_skirt.pickle', 'wb') as fw:
    pickle.dump(model, fw)

In [233]:
# 모델 불러오기
import pickle
with open('model_category_skirt.pickle', 'rb') as f:
    model_skirt = pickle.load(f)

In [234]:
def sample_recommendation(model, idx, user_ids):
    cnt = 1
    for user_id in user_ids:
        if cnt==1:
            scores = model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
        else:
            scores += model.predict(int(user_meta[user_meta.userId==user_id].index[0]), np.arange(len(cloth_meta)))
    
    top_items = idx[np.argsort(-scores)]
    
    sub_clothes = []

    for x in top_items[:10]:
        sub_clothes.append(x)
    
    rec_clothes = []
    
    for i in sub_clothes:
        rec_clothes.append(cloth_meta[cloth_meta.idx==i].newClothId.iloc[0])
    
    return rec_clothes

In [235]:
rec_size_top = sample_recommendation(model_skirt, cloth_meta.idx, real_user)
rec_size_top

[2017517424,
 150926840,
 1618698367,
 1907111403,
 1773210461,
 1173636175,
 2152938406,
 1618698368,
 1775973474,
 117363976]

In [236]:
real_user

[7389, 6361, 257]

In [238]:
cloth_meta.insert(5, 'what', 20, True)
user_meta.insert(4, 'what', 20, True)

In [239]:
cloth_meta = cloth_meta[["newClothId", "idx", 'what']]
cloth_meta

,newClothId,idx,what
0,161812522,1682,20
1,161812521,896,20
2,185310349,1662,20
3,1884867110,2304,20
4,1629303146,965,20
...,...,...,...
2649,21172562650,22,20
2650,21172562649,2,20
2651,21172562648,531,20
2652,23400702652,1441,20


In [240]:
cloth_meta.to_json('category_skirt_cloth_meta.json', orient='records', force_ascii=False)
user_meta.to_json('category_skirt_user_meta.json', orient='records', force_ascii=False)

In [241]:
user_meta

,userId,smallCategoryMiniskirt,smallCategoryMidi,smallCategoryLongskirt,what
0,0,0.0,0.0,1.0,20
1,1,0.5,0.0,0.5,20
2,2,1.0,0.0,0.0,20
3,3,1.0,0.0,0.0,20
4,4,1.0,0.0,0.0,20
...,...,...,...,...,...
8292,8292,0.0,0.0,1.0,20
8293,8293,0.0,0.0,1.0,20
8294,8294,1.0,0.0,0.0,20
8295,8295,1.0,0.0,0.0,20
